# Notebook 5: PCA para Detecção de Anomalias

## Objetivo
Usar a Análise de Componentes Principais (PCA) para detectar anomalias baseadas em **erro de reconstrução**.

## O Conceito
O PCA reduz a dimensionalidade dos dados projetando-os em eixos principais que capturam a maior variância.
- Dados normais seguem o padrão e são bem reconstruídos pelos primeiros componentes principais.
- Anomalias quebram a correlação padrão e perdem muita informação ao serem projetadas e depois re-projetadas (reconstruídas).
- **Alta perda de reconstrução = Anomalia.**

In [ ]:
!pip install -q pyod pandas matplotlib seaborn

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyod.models.pca import PCA
from pyod.utils.data import generate_data

plt.rcParams['figure.figsize'] = (10, 6)

## 1. Simulação: Sensores Industriais
Imagine uma fábrica com 3 sensores de temperatura que devem estar correlacionados (se o ambiente esquenta, todos sobem juntos).
- Normal: [20, 21, 20.5] ou [30, 31, 30.2]
- Anomalia: [20, 21, **45**] (Um sensor quebrou ou algo local pegou fogo).

In [ ]:
n_train = 500
n_test = 200
contamination = 0.05

# Gerando dados sintéticos (mais features = mais correlações a serem exploradas)
X_train, X_test, y_train, y_test = generate_data(n_train=n_train, n_test=n_test, n_features=5, contamination=contamination)

# Exemplo de 'correlação': Dados normais costumam viver num subespaço dimensional menor

## 2. Aplicando PCA
O PyOD implementa uma versão do PCA específica para outlier detection que calcula automaticamente a métrica de distância.

In [ ]:
# Inicializando o PCA
# n_components: quantos componentes manter? (O resto vira 'erro' / resíduo)
# Se n_features=5 e usarmos n_components=3, assumimos que os 2 ultimos são 'ruido' ou onde outliers se escondem.
clf_pca = PCA(n_components=3, contamination=contamination)
clf_pca.fit(X_train)

# Scores
y_train_pred = clf_pca.labels_
y_train_scores = clf_pca.decision_scores_

## 3. Explicabilidade (Variância Explicada)
É importante ver o quanto da informação os componentes escolhidos estão segurando.

In [ ]:
print(f"Variância explicada pelos 3 componentes: {sum(clf_pca.explained_variance_ratio_):.2%}")
# Se for muito alta (ex: 99%), significa que os outliers estão nesse 1% restante.

## 4. Avaliação

In [ ]:
from pyod.utils.data import evaluate_print

y_test_pred = clf_pca.predict(X_test)
y_test_scores = clf_pca.decision_function(X_test)

print("Resultados PCA:")
evaluate_print('PCA', y_test, y_test_scores)

### Visualizando os Scores de Reconstrução
Outliers têm scores substancialmente maiores porque não se alinham com os eixos principais.

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(y_test_scores, label='Score de Anomalia')
plt.axhline(clf_pca.threshold_, c='r', ls='--', label='Threshold')
plt.title('Score de Reconstrução para cada Amostra de Teste')
plt.ylabel('Erro de Reconstrução (Weighted)')
plt.legend()
plt.show()

## Conclusão
PCA é excelente para problemas lineares onde 'normal' significa 'correlacionado'. É muito rápido e computacionalmente barato.